### Домашняя работа 4

### Основное задание

Возьмите датасет https://www.kaggle.com/ajayrana/hymenoptera-data/kernels
 1. Обучите на нем модели ResNet 18 и VGG 16 с нуля (5-10 эпох)
 2. Обучите на нем модели ResNet 18 и VGG 16 с использованием FineTuning (5-10 эпох)
 3. Добавьте аугментацию данных к пункту 2

Сравните качество всех 3 полученных подходов

In [1]:
import torch
from torch import nn
import torchvision.models as models

import torchvision as tv
import time

import os

In [2]:
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
dev

device(type='cpu')

In [9]:
data_dir = '../dll/hymenoptera_data/'

In [10]:
BATCH_SIZE = 32

In [11]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': tv.transforms.Compose([
#         tv.transforms.RandomResizedCrop(224),
#         tv.transforms.RandomHorizontalFlip(),
#         tv.transforms.RandomVerticalFlip(),
        tv.transforms.CenterCrop(224),
        tv.transforms.ToTensor(),
        tv.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': tv.transforms.Compose([
        tv.transforms.Resize(256),
        tv.transforms.CenterCrop(224),
        tv.transforms.ToTensor(),
        tv.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: tv.datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


In [12]:
import torchvision.models as models
resnet18 = models.resnet18()
vgg16 = models.vgg16()

In [13]:
resnet18 = models.resnet18(pretrained=True)
vgg16 = models.vgg16(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\BEU_RU1/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\BEU_RU1/.cache\torch\hub\checkpoints\vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

#### 1.Обучите на нем модели ResNet 18 и VGG 16 с нуля (5-10 эпох)

### ResNet 18

In [14]:
model = models.resnet18()

In [15]:
model = model.to(dev)

In [16]:
def evaluate_accuracy(data_iter, net, dev):
    acc_sum, n = torch.Tensor([0]).to(dev), 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(dev), y.to(dev)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [17]:
def train(net, train_iter, test_iter, trainer, num_epochs, dev):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(dev), y.to(dev)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            print("Step. time since epoch: {:.3f}. Train acc: {:.3f}. Train Loss: {:.3f}".format(time.time() -  start,
                (y_hat.argmax(axis=1) == y).sum().item() / y.shape[0], l.item()))
        test_acc = evaluate_accuracy(test_iter, net, dev)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [18]:
lr, num_epochs = 0.001, 5
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, dataloaders_dict['train'], dataloaders_dict['val'], trainer, num_epochs, dev)

Step. time since epoch: 17.554. Train acc: 0.000. Train Loss: 229.013
Step. time since epoch: 25.908. Train acc: 0.531. Train Loss: 181.447
Step. time since epoch: 34.352. Train acc: 0.469. Train Loss: 168.221
Step. time since epoch: 41.885. Train acc: 0.375. Train Loss: 171.311
Step. time since epoch: 49.844. Train acc: 0.500. Train Loss: 115.404
Step. time since epoch: 58.297. Train acc: 0.406. Train Loss: 95.773
Step. time since epoch: 65.791. Train acc: 0.656. Train Loss: 53.171
Step. time since epoch: 71.158. Train acc: 0.400. Train Loss: 27.094
epoch 1, loss 4.2682, train acc 0.418, test acc 0.072, time 92.4 sec
Step. time since epoch: 28.371. Train acc: 0.031. Train Loss: 1209.573
Step. time since epoch: 36.451. Train acc: 0.469. Train Loss: 101.193
Step. time since epoch: 46.102. Train acc: 0.562. Train Loss: 74.255
Step. time since epoch: 54.066. Train acc: 0.438. Train Loss: 36.677
Step. time since epoch: 62.473. Train acc: 0.625. Train Loss: 31.955
Step. time since epoch: 70

In [19]:
# release CUDA
model.eval()
torch.cuda.empty_cache()

#### VGG 16

In [20]:
model = models.vgg16()

In [21]:
model = model.to(dev)

In [22]:
def evaluate_accuracy(data_iter, net, dev):
    acc_sum, n = torch.Tensor([0]).to(dev), 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(dev), y.to(dev)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [23]:
def train(net, train_iter, test_iter, trainer, num_epochs, dev):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(dev), y.to(dev)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            print("Step. time since epoch: {:.3f}. Train acc: {:.3f}. Train Loss: {:.3f}".format(time.time() -  start,
                (y_hat.argmax(axis=1) == y).sum().item() / y.shape[0], l.item()))
        test_acc = evaluate_accuracy(test_iter, net, dev)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [24]:
lr, num_epochs = 0.001, 5
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, dataloaders_dict['train'], dataloaders_dict['val'], trainer, num_epochs, dev)

Step. time since epoch: 68.127. Train acc: 0.000. Train Loss: 221.306
Step. time since epoch: 149.176. Train acc: 0.500. Train Loss: 1075.133
Step. time since epoch: 197.773. Train acc: 0.500. Train Loss: 108.346
Step. time since epoch: 283.396. Train acc: 0.531. Train Loss: 897.750
Step. time since epoch: 331.016. Train acc: 0.312. Train Loss: 188.278
Step. time since epoch: 381.541. Train acc: 0.656. Train Loss: 197.094
Step. time since epoch: 429.614. Train acc: 0.406. Train Loss: 187.530
Step. time since epoch: 457.164. Train acc: 0.500. Train Loss: 78.040
epoch 1, loss 12.1044, train acc 0.422, test acc 0.542, time 542.4 sec
Step. time since epoch: 53.242. Train acc: 0.562. Train Loss: 98.553
Step. time since epoch: 115.043. Train acc: 0.531. Train Loss: 30.052
Step. time since epoch: 161.898. Train acc: 0.562. Train Loss: 34.733
Step. time since epoch: 199.241. Train acc: 0.438. Train Loss: 54.199
Step. time since epoch: 236.036. Train acc: 0.594. Train Loss: 35.792
Step. time si

### 2.Обучите на нем модели ResNet 18 и VGG 16 с использованием FineTuning (5-10 эпох)

#### ResNet 18 pretrained

In [25]:
model = models.resnet18(pretrained=True)

In [26]:
model = model.to(dev)

In [27]:
## Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [28]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [29]:
model.fc = nn.Linear(in_features=512, out_features=2).to(dev)

In [30]:
print("Params to learn:")
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 fc.weight
	 fc.bias


In [31]:
def evaluate_accuracy(data_iter, net, dev):
    acc_sum, n = torch.Tensor([0]).to(dev), 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(dev), y.to(dev)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [32]:
def train(net, train_iter, test_iter, trainer, num_epochs, dev):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(dev), y.to(dev)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            print("Step. time since epoch: {:.3f}. Train acc: {:.3f}. Train Loss: {:.3f}".format(time.time() -  start,
                (y_hat.argmax(axis=1) == y).sum().item() / y.shape[0], l.item()))
        test_acc = evaluate_accuracy(test_iter, net, dev)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [33]:
lr, num_epochs = 0.001, 5
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, dataloaders_dict['train'], dataloaders_dict['val'], trainer, num_epochs, dev)

Step. time since epoch: 6.007. Train acc: 0.562. Train Loss: 22.929
Step. time since epoch: 8.148. Train acc: 0.594. Train Loss: 21.164
Step. time since epoch: 10.181. Train acc: 0.562. Train Loss: 19.349
Step. time since epoch: 12.168. Train acc: 0.562. Train Loss: 22.915
Step. time since epoch: 15.136. Train acc: 0.594. Train Loss: 20.688
Step. time since epoch: 18.230. Train acc: 0.812. Train Loss: 15.045
Step. time since epoch: 20.349. Train acc: 0.750. Train Loss: 18.141
Step. time since epoch: 21.725. Train acc: 0.850. Train Loss: 9.606
epoch 1, loss 0.6141, train acc 0.652, test acc 0.667, time 35.6 sec
Step. time since epoch: 6.489. Train acc: 0.688. Train Loss: 14.403
Step. time since epoch: 8.442. Train acc: 0.688. Train Loss: 16.306
Step. time since epoch: 10.576. Train acc: 0.688. Train Loss: 18.153
Step. time since epoch: 12.558. Train acc: 0.750. Train Loss: 16.469
Step. time since epoch: 14.478. Train acc: 0.969. Train Loss: 10.029
Step. time since epoch: 16.344. Train a

#### VGG 16 pretrained

In [34]:
model = models.vgg16(pretrained=True)

In [35]:
model = model.to(dev)

In [36]:
## Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [37]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [38]:
model.classifier[6] = nn.Linear(in_features=4096, out_features=2).to(dev)

In [39]:
print("Params to learn:")
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 classifier.6.weight
	 classifier.6.bias


In [40]:
def evaluate_accuracy(data_iter, net, dev):
    acc_sum, n = torch.Tensor([0]).to(dev), 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(dev), y.to(dev)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [41]:
def train(net, train_iter, test_iter, trainer, num_epochs, dev):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(dev), y.to(dev)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            print("Step. time since epoch: {:.3f}. Train acc: {:.3f}. Train Loss: {:.3f}".format(time.time() -  start,
                (y_hat.argmax(axis=1) == y).sum().item() / y.shape[0], l.item()))
        test_acc = evaluate_accuracy(test_iter, net, dev)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [42]:
lr, num_epochs = 0.001, 5
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, dataloaders_dict['train'], dataloaders_dict['val'], trainer, num_epochs, dev)

Step. time since epoch: 18.334. Train acc: 0.656. Train Loss: 23.186
Step. time since epoch: 31.569. Train acc: 0.781. Train Loss: 16.392
Step. time since epoch: 46.943. Train acc: 0.844. Train Loss: 13.506
Step. time since epoch: 60.508. Train acc: 0.875. Train Loss: 11.982
Step. time since epoch: 72.866. Train acc: 0.938. Train Loss: 7.274
Step. time since epoch: 85.267. Train acc: 0.906. Train Loss: 7.994
Step. time since epoch: 98.255. Train acc: 0.875. Train Loss: 9.825
Step. time since epoch: 105.977. Train acc: 1.000. Train Loss: 1.656
epoch 1, loss 0.3763, train acc 0.852, test acc 0.954, time 174.7 sec
Step. time since epoch: 15.217. Train acc: 1.000. Train Loss: 1.261
Step. time since epoch: 26.462. Train acc: 0.969. Train Loss: 2.318
Step. time since epoch: 37.865. Train acc: 0.875. Train Loss: 5.811
Step. time since epoch: 50.055. Train acc: 0.969. Train Loss: 3.183
Step. time since epoch: 62.608. Train acc: 1.000. Train Loss: 1.976
Step. time since epoch: 74.993. Train acc

### 3.Добавьте аугментацию данных к пункту 2

In [43]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': tv.transforms.Compose([
        tv.transforms.RandomResizedCrop(224),
        tv.transforms.RandomHorizontalFlip(),
        tv.transforms.RandomVerticalFlip(),
        tv.transforms.ToTensor(),
        tv.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': tv.transforms.Compose([
        tv.transforms.Resize(256),
        tv.transforms.CenterCrop(224),
        tv.transforms.ToTensor(),
        tv.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: tv.datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


#### ResNet 18 pretrained + aug

In [44]:
model = models.resnet18(pretrained=True)

In [45]:
model = model.to(dev)

In [46]:
## Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [47]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [48]:
model.fc = nn.Linear(in_features=512, out_features=2).to(dev)

In [49]:
print("Params to learn:")
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 fc.weight
	 fc.bias


In [50]:
def evaluate_accuracy(data_iter, net, dev):
    acc_sum, n = torch.Tensor([0]).to(dev), 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(dev), y.to(dev)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [51]:
def train(net, train_iter, test_iter, trainer, num_epochs, dev):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(dev), y.to(dev)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            print("Step. time since epoch: {:.3f}. Train acc: {:.3f}. Train Loss: {:.3f}".format(time.time() -  start,
                (y_hat.argmax(axis=1) == y).sum().item() / y.shape[0], l.item()))
        test_acc = evaluate_accuracy(test_iter, net, dev)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [52]:
lr, num_epochs = 0.001, 5
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, dataloaders_dict['train'], dataloaders_dict['val'], trainer, num_epochs, dev)

Step. time since epoch: 6.277. Train acc: 0.500. Train Loss: 30.117
Step. time since epoch: 8.236. Train acc: 0.469. Train Loss: 25.838
Step. time since epoch: 10.655. Train acc: 0.406. Train Loss: 25.473
Step. time since epoch: 13.001. Train acc: 0.500. Train Loss: 23.070
Step. time since epoch: 15.636. Train acc: 0.562. Train Loss: 24.052
Step. time since epoch: 17.982. Train acc: 0.562. Train Loss: 22.119
Step. time since epoch: 20.520. Train acc: 0.750. Train Loss: 20.064
Step. time since epoch: 21.774. Train acc: 0.450. Train Loss: 14.308
epoch 1, loss 0.7584, train acc 0.529, test acc 0.791, time 35.1 sec
Step. time since epoch: 5.560. Train acc: 0.656. Train Loss: 19.217
Step. time since epoch: 7.659. Train acc: 0.781. Train Loss: 16.738
Step. time since epoch: 9.612. Train acc: 0.719. Train Loss: 16.960
Step. time since epoch: 11.500. Train acc: 0.656. Train Loss: 18.656
Step. time since epoch: 13.347. Train acc: 0.719. Train Loss: 16.993
Step. time since epoch: 15.040. Train a

### VGG 16 pretrained + aug

In [53]:
model = models.vgg16(pretrained=True)

In [54]:
model = model.to(dev)

In [55]:
## Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [56]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [57]:
model.classifier[6] = nn.Linear(in_features=4096, out_features=2).to(dev)

In [58]:
print("Params to learn:")
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 classifier.6.weight
	 classifier.6.bias


In [59]:
def evaluate_accuracy(data_iter, net, dev):
    acc_sum, n = torch.Tensor([0]).to(dev), 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(dev), y.to(dev)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [60]:
def train(net, train_iter, test_iter, trainer, num_epochs, dev):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(dev), y.to(dev)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            print("Step. time since epoch: {:.3f}. Train acc: {:.3f}. Train Loss: {:.3f}".format(time.time() -  start,
                (y_hat.argmax(axis=1) == y).sum().item() / y.shape[0], l.item()))
        test_acc = evaluate_accuracy(test_iter, net, dev)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [61]:
lr, num_epochs = 0.001, 5
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, dataloaders_dict['train'], dataloaders_dict['val'], trainer, num_epochs, dev)

Step. time since epoch: 16.622. Train acc: 0.594. Train Loss: 20.171
Step. time since epoch: 27.230. Train acc: 0.688. Train Loss: 18.415
Step. time since epoch: 38.401. Train acc: 0.875. Train Loss: 12.666
Step. time since epoch: 48.837. Train acc: 0.969. Train Loss: 11.319
Step. time since epoch: 61.952. Train acc: 0.906. Train Loss: 9.464
Step. time since epoch: 74.581. Train acc: 0.875. Train Loss: 9.880
Step. time since epoch: 88.087. Train acc: 0.906. Train Loss: 7.500
Step. time since epoch: 95.043. Train acc: 0.950. Train Loss: 4.067
epoch 1, loss 0.3831, train acc 0.840, test acc 0.961, time 157.1 sec
Step. time since epoch: 16.435. Train acc: 0.906. Train Loss: 8.161
Step. time since epoch: 27.498. Train acc: 0.906. Train Loss: 7.181
Step. time since epoch: 38.357. Train acc: 0.906. Train Loss: 5.633
Step. time since epoch: 48.807. Train acc: 1.000. Train Loss: 2.979
Step. time since epoch: 59.759. Train acc: 1.000. Train Loss: 2.433
Step. time since epoch: 70.791. Train acc:

#### Сравните качество всех 3 полученных подходов
Качество обучения на тестовой и валидационной выборках в случае finetuning оказалось наилучшим. При этом предполагаем, что с учетом аугментации данных модель становится стабильнее.

### 4. Примените FineTuning ResNet 18 к FashionMnist. Удалось ли увидеть резкое увеличение качества?

In [62]:
BATCH_SIZE = 256

In [63]:
transoforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),
    tv.transforms.Resize((224,224)),
    tv.transforms.ToTensor()
])
train_dataset = tv.datasets.MNIST('.', train=True, transform=transoforms, download=True)
test_dataset = tv.datasets.MNIST('.', train=False, transform=transoforms, download=True)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [64]:
model = tv.models.resnet18(pretrained=True)

In [65]:
model = model.to(dev)

In [66]:
## Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [67]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [68]:
model.fc = nn.Linear(in_features=512, out_features=10).to(dev)

In [69]:
print("Params to learn:")
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 fc.weight
	 fc.bias


In [70]:
def evaluate_accuracy(data_iter, net, dev):
    acc_sum, n = torch.Tensor([0]).to(dev), 0
    for X, y in data_iter:
        X, y = X.to(dev), y.to(dev)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [71]:
def train(net, train_iter, test_iter, trainer, num_epochs, dev):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(dev), y.to(dev)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            print("Step. time since epoch: {:.3f}. Train acc: {:.3f}. Train Loss: {:.3f}".format(time.time() -  start,
                (y_hat.argmax(axis=1) == y).sum().item() / y.shape[0], l.item()))
        test_acc = evaluate_accuracy(test_iter, net, dev)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [72]:
lr, num_epochs = 0.001, 1
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, train_iter, test_iter, trainer, num_epochs, dev)

Step. time since epoch: 15.101. Train acc: 0.129. Train Loss: 640.037
Step. time since epoch: 31.788. Train acc: 0.102. Train Loss: 609.470
Step. time since epoch: 47.723. Train acc: 0.141. Train Loss: 584.580
Step. time since epoch: 63.548. Train acc: 0.199. Train Loss: 566.100
Step. time since epoch: 78.856. Train acc: 0.160. Train Loss: 558.196
Step. time since epoch: 94.336. Train acc: 0.250. Train Loss: 550.166
Step. time since epoch: 110.334. Train acc: 0.266. Train Loss: 519.706
Step. time since epoch: 125.844. Train acc: 0.293. Train Loss: 508.794
Step. time since epoch: 141.184. Train acc: 0.441. Train Loss: 478.146
Step. time since epoch: 156.429. Train acc: 0.410. Train Loss: 486.891
Step. time since epoch: 172.772. Train acc: 0.512. Train Loss: 457.443
Step. time since epoch: 188.440. Train acc: 0.516. Train Loss: 459.801
Step. time since epoch: 203.810. Train acc: 0.598. Train Loss: 431.280
Step. time since epoch: 219.836. Train acc: 0.637. Train Loss: 416.222
Step. time s

KeyboardInterrupt: 

хорошо обучается и выходит за 90-92% на сравнительном быстром периоде обучения, чем обычная сетка, но в определенный момент начинает переобучаться